In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import os
import time
from random import randint
from fuzzywuzzy import process

/home/frostburn/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def get_matches(query,choices,limit=4):
    results = process.extract(query,[choices],limit=limit)
    return results

In [3]:
print("download.default_directory=" + os.getcwd() + "/Downloads")

download.default_directory=/home/frostburn/Desktop/Web-Scraping/ResearchGate/Downloads


In [4]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.getcwd() + "/Downloads",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "headless":True
}
options.add_experimental_option('prefs', prefs)

In [5]:
start_page = "https://www.researchgate.net/search/authors?q"


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [6]:
def research_gate_scraper(record,driver):
    
    time.sleep(5)

    search_bar = driver.find_element_by_tag_name('input')
    search_bar.clear()
    search_bar.send_keys("\"" + record.faculty.strip() + "\" CSUN")
    search_bar.send_keys(Keys.ENTER)
    results = False

    if driver.find_elements_by_xpath('//*[@class="nova-o-stack__item"]/div'):
        results = True
    else:
        search_bar = driver.find_element_by_tag_name('input')
        search_bar.clear()
        search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if driver.find_elements_by_xpath('//*[@class="nova-o-stack__item"]/div'):
            results = True

    if results:

        results = False

        cards = driver.find_elements_by_xpath('//*[@class="nova-c-card__body nova-c-card__body--spacing-inherit"]')
        profile_links = driver.find_elements_by_link_text('View Profile')

        y = 0

        for card in cards:

            card = card.get_attribute('innerHTML')
            card_soup = BeautifulSoup(card, 'html.parser')

            name = card_soup.find('a',class_='nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare').text
            details = card_soup.find_all('li',class_='nova-e-list__item nova-v-person-item__info-section-list-item')

            if len(details) > 1:
                institution = details[0].text
                department = details[1].text
            elif len(details) > 0:
                institution = details[0].text
                department = ""
            elif len(details) == 0:
                institution = ""
                department = ""

            name_match = get_matches(name,record.faculty)
            print(name)
            institution_match = get_matches(institution,'California State University, Northridge')
            print(institution)
            department_match = get_matches(department,record.department)
            print(department)

            if (name_match[0][1] >= 85) and (institution_match[0][1] >= 90) and (department_match[0][1] >= 90):
                url = profile_links[y].get_attribute('href')
                results = True
            if results:
                 break    
            y += 1

    if results:
        print("Professor was found!")
        citation_collector(record,url)
    else:
        print("Professor was added to no results file.")
        f = open("research_gate_no_results.csv", "a")
        f.write(record.faculty + ",")
        f.write(record.college + ",")
        f.write(record.department + ",")
        f.write(record.url + "\n")
        f.close()


In [7]:
def citation_collector(record,url):
    
    bio = ""
    dept = ""
    personal = ""
    academic = ""
    research = ""
    orcid = ""
    scopus = ""     
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    driver.get(url)
    try:
        #Attempt to find the total number of research items for a given professsor 
        num_research_items = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit"]')
        num_research_items = num_research_items.text
        num_research_items = int(num_research_items)
        print("Number of research items:",num_research_items)
    except NoSuchElementException:
        num_research_items = 0


    paper_types = []
    urls = []
        
    # grab the research interests
    interests = driver.find_elements_by_xpath('//*[@id="about"]/div/div/div[3]/div/div/div[2]/div/div[2]/div/div/div/div/div')
    interests = [item.text for item in interests]
    while '' in interests:
        interests.remove('')
    for interest in interests:
        research += interest + " "

    if num_research_items > 0:
        # Retrieve all the urls of the first page
        research_a_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
        urls = [tag.get_attribute("href") for tag in research_a_tags]
        titles = [tag.text for tag in research_a_tags]
        paper_types = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
        paper_types = [item.text for item in paper_types]
        num_research_items -= 100
        current_url = driver.current_url

    page_count = 2
    while num_research_items > 0:
        driver.get(current_url + '/' + str(page_count))
        temp_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
        for tag in temp_tags:
            urls.append(tag.get_attribute("href"))
            titles.append(tag.text)
        temp_papers = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
        for paper in temp_papers:
            paper_types.append(paper.text)
        num_research_items -= 100
        page_count += 1
    
    if len(paper_types) > 0:
        while "Full-text available" in paper_types:
            paper_types.remove("Full-text available")

    profile = open("research_gate_profiles.csv", "a")
    profile.write(record.faculty.strip() + "|" + record.college + "|")
    profile.write(record.department + "|" + bio + "|" + personal + "|")
    profile.write(academic + "|" + research + "|")
    profile.write(orcid + "|" + scopus + "\n")
    profile.close()

    if len(urls) > 0:
        i = 0
        for url in urls:

            # experimenting handling alerts      this might still have merit
    #         if alert(driver):
    #             alert(driver).dismiss()

            print(i)

            time.sleep(3)

            citation_text = ""
            title = ""
            abstract = ""
            book_title = ""
            journal_title = ""
            date = ""
            collabs = ""
            publisher = ""
            edition = ""
            series = ""
            volume = ""
            issue = ""
            chapter = ""
            pages = ""
            doi = ""
            issn = ""
            isbn = ""
            handle = ""

            #time.sleep(randint(5,10))
            driver.get(url)

            # check for the DOI
            if driver.find_elements_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]'):
                # Grab the DOI
                meta = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]')
                doi = meta.text.split(" ")
                if len(doi) > 1:
                    if '\n' in doi[1]:
                        doi = doi[1].split("\n")
                        doi = doi[0]
                    else:
                        doi = doi[1]
                else:
                    doi = ""

            # check for the abstract
            if driver.find_elements_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]'):
                # get the abstract
                abstract = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]')    
                abstract = abstract.text

            # then we'll get the collaborators
            collab = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/ul/li/div/div/div/div/div[2]/div/div/div/div/div/a')
            collabs = ""
            for person in collab:
                collabs += person.text


            # Attempt to get the date
            if driver.find_elements_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]'):
                date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
                date = date.text

            #download the citatation
            #//*[@id="lite-page"]/main/section/section[1]/div[1]/a
            download_link = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section/div/a')
            download_link.click()

            # click the text button
            text_radio_btn = driver.find_element_by_xpath("//span[contains(.,' Plain Text')]")
            text_radio_btn.click()

            # download the citation file
            download = driver.find_element_by_xpath('//button[@class="nova-c-button nova-c-button--align-center nova-c-button--radius-m nova-c-button--size-m nova-c-button--color-blue nova-c-button--theme-solid nova-c-button--width-auto"]')
            download.click()

            time.sleep(3)

            #driver.close()


            # now that file is downloaded, need to open the file

            # here I grab the file name; the trick to this is making sure that its the only file in downloads
            citation_file = os.listdir(os.getcwd() + "/Downloads")[0]

            # grab the citation
            # i can probably get the name of the journal from the citation if thats something thats all over
            f = open("Downloads/"+citation_file, "r")
            citation_text = f.read()
            f.close()
            # delete it
            os.remove("Downloads/"+citation_file)

            citation_text


            pieces = citation_text.split('.')
            parts = []
            for piece in pieces:
                parts.append(piece.strip())
            #print(parts[3],",",parts[4])

            citations = open("research_gate_citations.csv", "a")
            citations.write(record.faculty.strip() + "|" + citation_text + "|" + titles[i] + "|" + paper_types[i] + "|")
            citations.write(abstract + "|" + book_title + "|" + journal_title + "|" + date + "|")
            citations.write(collabs + "|" + publisher + "|" + edition + "|" + series + "|")
            citations.write(volume + "|" + issue + "|" + chapter + "|" + pages + "|")
            citations.write(doi + "|" + issn + "|" + isbn + "|" + handle + "|" + url + "\n")
            citations.close()
            i+=1
    driver.close()
            

### The goal is to run all_faculty.csv through this scraper

In [8]:
all_faculty = pd.read_csv("../Data Sets/all_faculty.csv")
print(all_faculty.shape[0])
print(all_faculty.columns)
all_faculty.head()

2579
Index(['faculty', 'college', 'department', 'url'], dtype='object')


,faculty,college,department,url
0,Keji Chen,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
1,Michael E. Doron,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
2,Rafi Efrat,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
3,Monica Gianni,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
4,Young-Won Her,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...


In [9]:
all_faculty.iloc[2134]

faculty                                      Gilberto E. Flores
college                                        Science and Math
department                                Department of Biology
url            https://www.csun.edu/science-mathematics/biol...
Name: 2134, dtype: object

In [19]:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(4)
try:
    research_gate = "https://www.researchgate.net/search/authors?q"
    driver.get(research_gate)
    for i in range(2526, all_faculty.shape[0]):
        print(i,":",all_faculty.iloc[i].faculty)
        research_gate_scraper(all_faculty.iloc[i],driver)
finally:
    driver.close()

2526 : Victoria Nichols 


Victoria Nichols
Indiana University Northwest
School of Nursing
mª Victoria
Universidad de Burgos

Juan Ventura Victoria
University of Oviedo
Department of Business Administration
Patricia Victoria
Catholic University of the Most Holy Conception
Departamento de Ingeniería Civil
Sónia Victória
University of Cape Verde
Departamento de Ciência e Tecnologia (DCT)
Abad san martin Victoria
Universidad de Burgos
Department of Electromechanical Engineering 
Katie Victoria
Aberystwyth University
Institute of Biological, Environmental and Rural Sciences
Stephanie Victoria
Hochschule Heilbronn
medizinische Informatik
Olajumoke Victoria
Federal University Oye-Ekiti
Department of Microbiology
Ruby Victoria
University of California, Davis
Department of Animal Science
Professor was added to no results file.
2527 : Matthew Parke 


Tim Parke
NHS Greater Glasgow and Clyde
Department of Emergency
Amy l Parke
Maryville University
Department of Nursing
Laura G. Parke
University of Utah
Department of Chemistry
Lydia Parke
Salus University
Research
Amy Parke
Queen's University Belfast
Centre for Biomedical Sciences Education
Simon Parke
Derby Hospitals NHS Foundation Trust
Giant coronary artery aneurysms in a 12-week-old infant with incomplete Kawasaki...
Heather Parke
Durham University
School of Applied Social Sciences
Brendan Parke
Ulster University
School of Education
Morgan Parke
University of Minnesota Twin Cities
Department of Neuroscience
Sara Parke


Professor was added to no results file.
2528 : Daniel Parsons 


Daniel Parsons
International University of Japan
Center for Language Education and Research
Daniel Parsons
Adams State University
Art
Daniel Parsons
University of Leeds
Section of Food Chemistry and Biochemistry
Daniel Parsons
Flinders University

Daniel Parsons
Massachusetts College of Pharmacy and  Health Sciences
School of Optometry
Daniel Parsons
Kwansei Gakuin University
School of Science and Technology
Daniel Parsons
University of Canberra
Faculty of Arts and Design
Daniel Parsons
Tensile strength of pine needles and their feasibility as reinforcement in compo...

Daniel Parsons
Kingston University
Department of Pathology
Daniel Roy Parsons
University of Hull
Department of Geography, Environment and Earth Sciences
Professor was added to no results file.
2529 : Rebecca Pena 
Rebecca Pena
University of South Florida
Department of Psychology
Rebecca Pena
University of Texas at Arlington
College of Nursing
Klein Rebecca
Cheetah Conservation Botswana
Status Report for the Cheetah in B

Judy Grant
University of Kansas
Obstetric Delivery
Sabrina Judy
The Catholic University of America
School of Nursing
Ryan Judy
University of Pittsburgh
School of Medicine
Robert Judy
Stephen F. Austin State University
College of Education
Scott Judy
Carnegie Mellon University
Computer Science Department
Jack Judy
Sparrow Health System
Emergency Department
Diana Judy
Los Angeles City College
Behavioral Psychology
Wesley Judy
University of Toledo
Department of Medicine
Brendan F Judy
Thomas Jefferson University
Small Portable Interchangeable Imager of Fluorescence for Fluorescence Guided Su...
Gatward Judy


Professor was added to no results file.
2532 : Robert Plotke 


...

Abby Simonds
Western Kentucky University
School of Nursing
Katharine Simonds
Oxford Health NHS Foundation Trust
Department of Physiotherapy
Alisa Simonds
University of East London
Department of Psychology
Anita Simonds
Royal Brompton and Harefield NHS Foundation Trust, London SW3 6NP
Academic Dept of Sleep & Medicine
Henry Simonds
Vermont College of Fine Arts
Visual Art
Amy Simonds
University of Chester

Karen Simonds
University of Strathclyde
School of Psychological Sciences and Health
Brian J. Simonds
National Institute of Standards and Technology, Boulder, CO
Silicon
Caitlyn Simonds
University of Florida
Department of Mechanical and Aerospace Engineering
Kohl Simonds
San Diego State University
Department of Finance
Professor was added to no results file.
2570 : Jasmin Tobar  
Jennifer Jasmin
The College of New Jersey
Department of Nursing
Elsie Jasmin
Long Island University
Department of Biology
Louise Jasmin
Fielding Graduate University
School of Human and Organizational Developmen

Joseph Wiltberger
University of California, San Diego
Center For Comparative Immigration Studies
Georg Wiltberger
University of Leipzig
Laparoscopic
Lars Wiltberger
Leiden University
Security Studies
Michael Wiltberger
National Center for Atmospheric Research
High Altitude Observatory
Colleen Wiltberger
University of North Florida
Department of Biology
Anthony Joseph
The University of Sydney
Finite Element Modeling
Emily Joseph
City University of New York - John Jay College of Criminal Justice
Department of Psychology
Nephtalie Eva Joseph


Rita Joseph
Flinders University
College of Education, Psychology and Social Work
Caryl Len Joseph
University of the West Indies at Cave Hill, Barbados
Faculty of Social Sciences
Professor was added to no results file.
2572 : An Yountae  . 


Kang Yountae


Kim Yountae
Heinrich-Heine-Universität Düsseldorf
hhu
Yountae An
Lebanon Valley College
Religion
Yountae An
Drew University
Graduate Division of Religion
Yountae Kim
Baekseok University
Department of Divinity
Alvin Csunderlik
West University of Timisoara
Department of Psychology
Marlene Csunyo
All Children's Hospital
NICU
Marlene Csunyo
St. Petersburg College
Department of Health and Nursing
Professor was added to no results file.
2573 : Arturo Zepeda  
Arturo Zepeda
Universidad Michoacana de San Nicolás de Hidalgo
Faculty of Economics “Vasco de Quiroga”
Nicolas Arturo
Arturo Prat University
Departamento de Ciencias Químicas y Farmacéuticas
Karlo Arturo
Universidad Nacional Agraria La Molina
Department of Nutrition (DAN)
Ian A Arturo
The University of Western Ontario
Department of Earth Sciences
Perrusquía Arturo
Autonomous University of Mexico City
Ciencias Sociales
Julio Arturo
Universidad Pedagógica Experimental Libertador
Ingenieria Petroquímica
Reyes Arturo
Institut

Tomas Sandor
Moscow State Institute of International Relations
Faculty of International Relations
Balázs Sándor
First integrals and integrating factors of second order autonomous systems

Péter Sándor
Stony Brook University

Marilyn Sandor
Naples Pediatric Dentistry
Pediatric Dentistry
M. Kay Sandor
University of Texas Medical Branch at Galveston
School of Nursing
Kecskeméti Sándor
Szent István University, Godollo
Vegetable and Mushroom Growing
Bacso Sandor
University of Debrecen
Department of Geometry
Paul Sandor
University of Toronto
Department of Psychiatry
Csaba Sandor
Budapest University of Technology and Economics
Department of Stochastics
Adam Miklos Sandor
University of Pennsylvania
Department of Pathology and Laboratory Medicine
Professor was added to no results file.
2575 : Kurt Saunders 
Kurt Saunders
California State University, Northridge
Department of Business Law
Professor was found!
Number of research items: 26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


Gregory Young
Antioch University

Gregory Young
Montana State University
Music
Gregory Young
University of Colorado Boulder
Department of Political Science
Gregory Young
The University of the West Indies at Mona
Department of History and Archaeology
Gregory Young
University of Bath
Department of Mechanical Engineering
Gregory Young
CCI
Collagen
Aaron Gregory Young
University of California, Berkeley
Department of City and Regional Planning
Norman Gregory Young
California State Polytechnic University, Pomona
Department of Finance, Real Estate, & Law
Justin Gregory Young
Kettering University
Department of Industrial and Manufacturing Engineering
Sean Gregory Young
University of Arkansas for Medical Sciences
Department of Environmental and Occupational Health
Professor was added to no results file.
